In [1]:
import logging
from websocket_server import WebsocketServer
import json

import numpy as np
import math

def new_client(client, server):
	print("connected")
    
def client_disconnected(client, server):
    print("disconnected")
    exit(1)
    
def _check_mode(msg):
    return  "Auto" if msg and msg[:2] == '42' else "Manual"

def _parse_telemetry(msg):
    msg_json = msg[2:]
    parsed = json.loads(msg_json)
    
    msg_type = parsed[0]
    assert msg_type == 'telemetry' , "Invalid message type {}".format(msg_type) 
    
    # Telemetry values
    values = parsed[1]
    
    return values


In [5]:
def msg_received(client, server, message):
    if _check_mode(message) is 'Auto':
        telemetry  = _parse_telemetry(message)
            
        psi = telemetry['psi']
        ptsx = telemetry['ptsx']
        ptsy = telemetry['ptsy']
        speed = telemetry['speed'] * 0.44704 #MPH to m/s
        steering = telemetry['steering_angle']
        throttle = telemetry['throttle']
        x = telemetry['x']
        y = telemetry['y']
        
        for i in range(len(ptsx)):
            delta_x = ptsx[i]-x
            delta_y = ptsy[i]-y
            transformed_x = (delta_x * math.cos(-psi) - delta_y * math.sin(-psi))
            transformed_y = (delta_x * math.sin(-psi) + delta_y * math.cos(-psi))
        
        coeffs = np.polyfit(transformed_x, transformed_y, 3)
        
        cte = np.polyval(coeffs, 0)
        
        epsi = -math.atan(coeffs[1])
        
        Lf = 2.67;
        dt = 0.1;
        
        pred_px = 0.0 + v * dt
        pred_py = 0.0
        pred_psi = 0.0 + v * -steering / Lf * dt
        pred_v = v + throttle * dt
        pred_cte = cte + v * math.sin(epsi) * dt
        pred_epsi = epsi + v * -steering / Lf * dt
        
        states = [pred_px, pred_py, pred_psi, pred_v, pred_cte, pred_epsi]
        
        variables = method_call(states)
        
        steering = -variables[0]/(math.radians(25)*Lf)
        throttle = variables[1]
        #Implement your model predictive control here.
    else:
        print("Falling back to manual mode")

In [ ]:
server = WebsocketServer(4567, host='127.0.0.1', loglevel=logging.INFO)
server.set_fn_new_client(new_client)
server.set_fn_message_received(msg_received)
server.set_fn_client_left(client_disconnected)
server.run_forever()

INFO:websocket_server.websocket_server:Listening on port 4567 for clients..


connected
------------------------------------------
42["telemetry",{"ptsx":[-32.16173,-43.49173,-61.09,-78.29172,-93.05002,-107.7717],"ptsy":[113.361,105.941,92.88499,78.73102,65.34102,50.57938],"psi_unity":4.120315,"psi":3.733667,"x":-40.62008,"y":108.7301,"steering_angle":0,"throttle":0,"speed":5.422823E-06}]
------------------------------------------
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
-40.62008
------------------------------------------
2
------------------------------------------
Falling back to manual mode


In [1]:
import tensorflow as tf

In [2]:
a = tf.placeholder(tf.float32, shape = [3], name = 'coeffs')

x = tf.Variable(tf.zeros([1]), name = 'x')

cost = a[2] + a[1]*x + a[0]*x**2


In [3]:
opt = tf.train.GradientDescentOptimizer(0.01)

In [4]:
minimize_operation = opt.minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(200):
        x_val, cost_val, _ = sess.run([x, cost, minimize_operation], feed_dict={a: [1, 5, 6]})
        print(x_val, cost_val)

[-0.05] [6.]
[-0.09899999] [5.7525]
[-0.14701998] [5.274701]
[-0.19407958] [5.051217]
[-0.24019799] [4.836677]
[-0.285394] [4.630725]
[-0.32968614] [4.4330187]
[-0.3730924] [4.243231]
[-0.41563058] [4.061045]
[-0.45731798] [3.886159]
[-0.49817163] [3.7182817]
[-0.5382082] [3.557134]
[-0.577444] [3.402448]
[-0.61589515] [3.2539656]
[-0.65357727] [3.1114404]
[-0.69050574] [2.9746346]
[-0.72669566] [2.84332]
[-0.76216173] [2.717278]
[-0.7969185] [2.596298]
[-0.8309801] [2.4801786]
[-0.8643605] [2.3687253]
[-0.89707327] [2.2617526]
[-0.9291318] [2.1590815]
[-0.9605492] [2.0605402]
[-0.9913382] [1.965964]
[-1.0215114] [1.8751941]
[-1.0510812] [1.7880797]
[-1.0800595] [1.704474]
[-1.1084583] [1.624237]
[-1.1362891] [1.5472342]
[-1.1635634] [1.4733362]
[-1.1902921] [1.4024189]
[-1.2164863] [1.3343636]
[-1.2421566] [1.269056]
[-1.2673135] [1.2063857]
[-1.2919672] [1.1462475]
[-1.3161278] [1.0885402]
[-1.3398052] [1.0331662]
[-1.3630091] [0.9800327]
[-1.3857489] [0.9290495]
[-1.4080338] [0.8801

In [3]:
from tensorflow.contrib.opt import ScipyOptimizerInterface

a = tf.placeholder(tf.float32, shape = [3], name = 'coeffs')

x = tf.Variable(tf.zeros([1]), name = 'x')

cost = a[2] + a[1]*x + a[0]*x**2

optimizer = ScipyOptimizerInterface(
            cost, var_to_bounds = {x:(-1,65545)})

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    optimizer.minimize(session, feed_dict={a: [1, 5, 6]})
    x_val, cost_val = session.run([x, cost], feed_dict={a: [1, 5, 6]})
    print('x={}, cost={}'.format(x_val, cost_val))

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 2.000000
  Number of iterations: 1
  Number of functions evaluations: 2
x=[-1.], cost=[2.]


In [ ]:
from tensorflow.contrib.opt import ScipyOptimizerInterface

delta_t = tf.constant(0.05)

Lf = tf.constant(2.67)

## Define independent variables.
x_t0 = tf.get_variable("x_t", initializer= _known_value_)
y_t0 = tf.get_variable("x_t", initializer= _known_value_)
v_t0 = tf.get_variable("v_t", initializer= _known_value_)
psi_t0 = tf.get_variable("w_t", initializer= _known_value_)
cte_t0 = tf.get_variable("cte_t", initializer= _known_value_)
epsi_t0 = tf.get_variable("cw_t", initializer= _known_value_)
delta_t0 = tf.get_variable("delta_t0", initializer= _known_value_)
a_t0 = tf.get_variable("a_t0", initializer= _known_value_)

## We need to write dependencies in terms of initialized values of
## our dependent values. This is an ugly requirement of TensorFlow
## that we have to live with. 
## See https://www.tensorflow.org/programmers_guide/variables
## for details. 
x_t0_val = x_t0.initialized_value()
y_t0_val = y_t0.initialized_value()
v_t0_val = v_t0.initialized_value()
psi_t0_val = psi_t0.initialized_value()
cte_t0_val = cte_t0.initialized_value()
epsi_t0_val = epsi_t0.initialized_value()
delta_t0_val = delta_t0.initialized_value()
a_t0_val = a_t0.initialized_value()

## Define dependent variable using initializer parameter to connect x_t1 to values
## of x_t0, v_t0 and psi_t0
x_t1 = tf.get_variable("x_t1", initializer= x_t0_val + v_t0_val * tf.cos(psi_t0_val) * delta_t)
y_t1 = tf.get_variable("y_t1", initializer= y_t0_val + v_t0_val * tf.sin(psi_t0_val) * delta_t) 
psi_t1 = tf.get_variable("psi_t1", initializer= psi_t0_val + v_t0_val *delta_t0_val * delta_t/Lf) 
v_t1 = tf.get_variable("v_t1", initializer= v_t0_val + a_t0_val * delta_t) 
cte_t1 = tf.get_variable("x_t1", initializer= cte_t0_val + v_t0_val * tf.sin(psi_t0_val) * delta_t) 
epsi_t1 = tf.get_variable("psi_t1", initializer= epsi_t0_val + v_t0_val *delta_t0_val * delta_t/Lf) 


In [ ]:
def class MPC:
    def _init_(self, coeffs):
        
        delta_t = tf.constant(0.1)
        Lf = tf.constant(2.67)
        N = tf.constant(10)
        v_ref = tf.constant(40)
        power = tf.constant(2)
        
        x_0 = tf.placeholder(tf.float32, shape = [1], name = 'x0')
        y_0 = tf.placeholder(tf.float32, shape = [1], name = 'y0')        
        psi_0 = tf.placeholder(tf.float32, shape = [1], name = 'psi0')        
        v_0 = tf.placeholder(tf.float32, shape = [1], name = 'v0')        
        cte_0 = tf.placeholder(tf.float32, shape = [1], name = 'cte0')        
        epsi_0 = tf.placeholder(tf.float32, shape = [1], name = 'epsi0')
        coeffs = tf.placeholder(tf.float32, shape = [4], name = 'x0')
        
        self.delta_0 = tf.Variable(tf.zeros([1]), name = 'delta0')
        self.a_0 = tf.Variable(tf.zeros([1]), name = 'a0')
        
        self.cost = 2000*tf.pow(cte_0,power)+2000*tf.pow(epsi_0,power)+tf.pow(v_0-v_ref,power)
                    +5*tf.pow(self.a_0,power)+5*tf.pow(self.delta_0,power)
        
        for i in range(N):
            
            f0 = coeffs[0] + (coeffs[1] * x_0) + (coeffs[2] * x_0 * x_0) + (coeffs[3] * x_0 * x_0 * x_0)
            psides0 = tf.atan((3 * coeffs[3]* x_0 * x_0) + (2 * coeffs[2] * x_0) + coeffs[1])
            x_1 = x_0 + v_0*tf.cos(psi_0)*delta_t
            y_1 = y_0 + v_0*tf.sin(psi_0)*delta_t
            psi_1 = psi_0 + v_0 * delta_0 * delta_t/Lf
            v_1 = v_0 + a_0*delta_t
            cte_1 = f0 - y_0 + v_0*tf.sin(epsi_0)*delta_t
            epsi_1 = (psi_0 - psides0) + v_0 * delta_0 * delta_t/Lf
            
            self.cost += 2000*tf.pow(cte_1,power)+2000*tf.pow(epsi_1,power)+tf.pow(v_1-v_ref,power)
                    +5*tf.pow(self.a_0,power)+5*tf.pow(self.delta_0,power)
            
    def solve():
              